In [ ]:
import json

def load_highergov_opportunities(filepath):
    """
    Load the HigherGov API file and extract opportunity identifiers.
    Each opportunity is represented as a set containing:
      - source_id
      - source_id_version
    Returns a list of sets.
    """
    with open(filepath, "r") as f:
        data = json.load(f)
        # Determine if data is wrapped in a dict or is a list of records.
        if isinstance(data, dict):
            records = data.get("results", [])
        elif isinstance(data, list):
            records = data
        else:
            records = []
    opportunities = []
    for record in records:
        ids = set()
        if "source_id" in record:
            ids.add(record["source_id"])
        if "source_id_version" in record:
            ids.add(record["source_id_version"])
        opportunities.append(ids)
    return opportunities

def load_results_opportunities(filepath):
    """
    Load the results JSON file and extract opportunity identifiers.
    Each opportunity is represented as a set containing:
      - solicitation_id (if it exists)
      - notice_id (if it exists)
      - solicitationNumber from any history records (if available)
    Returns a list of sets.
    """
    with open(filepath, "r") as f:
        data = json.load(f)
    opportunities = []
    for record in data:
        ids = set()
        if "solicitation_id" in record:
            ids.add(record["solicitation_id"])
        if "notice_id" in record:  # Added to catch identifiers like "1e3bb70baf844632878847c776199d92".
            ids.add(record["notice_id"])
        if "history" in record:
            for hist in record["history"]:
                if "solicitationNumber" in hist:
                    ids.add(hist["solicitationNumber"])
        opportunities.append(ids)
    return opportunities

def find_match(opportunity, other_opportunities):
    """
    Given a set representing an opportunity, check if there is any opportunity in other_opportunities
    which has a non-empty intersection with it.
    If so, the opportunity is considered to have a match.
    """
    for other in other_opportunities:
        if opportunity.intersection(other):
            return True
    return False

def main():
    highergov_file = "wnzTPS5NfNK5vVqRhbQ9i_highergov.json"
    results_file = "wnzTPS5NfNK5vVqRhbQ9i_supabase.json"

    # Load opportunities from both files.
    highergov_ops = load_highergov_opportunities(highergov_file)
    results_ops = load_results_opportunities(results_file)
    print("Highergov opportunities:", highergov_ops)
    
    # Determine mismatches by checking each opportunity against the other file.
    mismatch_records_highergov = [op for op in highergov_ops if not find_match(op, results_ops)]
    mismatches_highergov = len(mismatch_records_highergov)
    
    mismatch_records_results = [op for op in results_ops if not find_match(op, highergov_ops)]
    mismatches_results = len(mismatch_records_results)
    
    total_mismatches = mismatches_highergov + mismatches_results

    print("----- Mismatch Report -----")
    print("Number of entries from highergov_api_full_data.json:", len(highergov_ops))
    print("Number of entries from results.json:", len(results_ops))
    print("Number of opportunities in highergov_api_full_data.json with no match in results.json:", mismatches_highergov)
    print("Number of opportunities in results.json with no match in highergov_api_full_data.json:", mismatches_results)
    print("Total mismatches:", total_mismatches)
    
    # Print out the mismatch details
    print("\nDetailed Mismatches:")
    print("Mismatched opportunities from highergov_api_full_data.json:")
    for op in mismatch_records_highergov:
        print(op)
    
    print("\nMismatched opportunities from results.json:")
    for op in mismatch_records_results:
        print(op)

if __name__ == "__main__":
    main()

In [ ]:
import csv
import json

def load_csv_opportunities(filepath):
    """
    Load the contract opportunity CSV file and extract opportunity identifiers.
    Each opportunity is represented as a set containing:
      - source_id
      - source_id_version
    Returns a list of sets.
    """
    opportunities = []
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            #print(row)
            ids = set()
            # Check and add identifier fields if present and non-empty.
            if 'Solicitation ID' in row and row['Solicitation ID']:
                ids.add(row['Solicitation ID'])
            if 'Solicitation Title' in row and row['Solicitation Title']:
                ids.add(row['Solicitation Title'])
            if '\ufeffNotice ID' in row and row['\ufeffNotice ID']:
                #print("catch")
                ids.add(row['\ufeffNotice ID'])
            opportunities.append(ids)
    return opportunities

def load_results_opportunities(filepath):
    """
    Load the supabase JSON file and extract opportunity identifiers.
    Each opportunity is represented as a set containing:
      - solicitation_id (if it exists)
      - notice_id (if it exists)
      - solicitationNumber from any history records (if available)
    Returns a list of sets.
    """
    with open(filepath, "r") as f:
        data = json.load(f)
    opportunities = []
    for record in data:
        ids = set()
        if "solicitation_id" in record and record["solicitation_id"]:
            ids.add(record["solicitation_id"])
        if "notice_id" in record and record["notice_id"]:
            ids.add(record["notice_id"])
        if "title" in record and record["title"]:
            ids.add(record["title"])
        if "history" in record:
            for hist in record["history"]:
                if "solicitationNumber" in hist and hist["solicitationNumber"]:
                    ids.add(hist["solicitationNumber"])
        opportunities.append(ids)
    return opportunities

def find_match(opportunity, other_opportunities):
    """
    Given a set representing an opportunity, check if there is any opportunity in other_opportunities
    that has a non-empty intersection with it.
    If so, the opportunity is considered to have a match.
    """
    for other in other_opportunities:
        if opportunity.intersection(other):
            return True
    return False

def main():
    csv_file = "contract_opportunity-03-17-25-18-48-13.csv"
    results_file = "wnzTPS5NfNK5vVqRhbQ9i_results.json"

    # Load opportunities from both files.
    csv_ops = load_csv_opportunities(csv_file)
    results_ops = load_results_opportunities(results_file)
    #print("CSV opportunities:", csv_ops)
    
    # Determine mismatches by checking each opportunity against the other file.
    mismatch_records_csv = [op for op in csv_ops if not find_match(op, results_ops)]
    mismatches_csv = len(mismatch_records_csv)
    
    mismatch_records_results = [op for op in results_ops if not find_match(op, csv_ops)]
    mismatches_results = len(mismatch_records_results)
    
    total_mismatches = mismatches_csv + mismatches_results

    print("----- Mismatch Report -----")
    print("Number of entries from CSV file:", len(csv_ops))
    print("Number of entries from results.json:", len(results_ops))
    print("Opportunities in highergov CSV with no match in supabase:", mismatches_csv)
    print("Opportunities in supabase with no match in highergov CSV:", mismatches_results)
    print("Total mismatches:", total_mismatches)
    
    # Print out the mismatch details
    print("\nDetailed Mismatches:")
    print("Mismatched opportunities from CSV file:")
    for op in mismatch_records_csv:
        print(op)
    
    print("\nMismatched opportunities from results.json:")
    for op in mismatch_records_results:
        print(op)

if __name__ == "__main__":
    main()

In [24]:
import os
import re
import json
from typing import List, Dict, Optional
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime, timezone

def get_filtered_notices(
    supabase: Client,
    active: bool = True,
    include_naics: Optional[List[str]] = None,
    exclude_naics: Optional[List[str]] = None,
    include_solicitation_types: Optional[List[str]] = None,
    exclude_solicitation_types: Optional[List[str]] = None,
    include_psc: Optional[List[str]] = None,
    exclude_psc: Optional[List[str]] = None,
    include_set_aside_ids: Optional[List[str]] = None,  # new filter: include by set_aside_id
    exclude_set_aside_ids: Optional[List[str]] = None,  # new filter: exclude by set_aside_id
    keyword_query: Optional[str] = None
) -> List[Dict[str, any]]:
    """
    Retrieve rows from the 'notices' table applying the provided filters and paginating through 
    all the available data.
    """
    all_notices = []
    limit = 1000  # Batch size for pagination.
    offset = 0

    while True:
        # Build the base query with embedded joins.
        query = supabase.from_("notices").select("""
            *,
            naics_details:naics!naics_id(*),
            psc_details:psc!psc_id(*),
            setasides_details:setasides!set_aside_id(*),
            solicitations_details:solicitations!solicitation_id(*),
            addresses_details:addresses!organization_address_key(*),
            organization_details:organizations!Notices_organization_key_fkey(*),
            organization_level_1_details:organizations!Notices_organization_level_1_key_fkey(*),
            organization_level_2_details:organizations!Notices_organization_level_2_key_fkey(*),
            organization_level_3_details:organizations!Notices_organization_level_3_key_fkey(*),
            organization_level_4_details:organizations!Notices_organization_level_4_key_fkey(*),
            organization_level_5_details:organizations!Notices_organization_level_5_key_fkey(*),
            organization_level_6_details:organizations!Notices_organization_level_6_key_fkey(*),
            organization_level_7_details:organizations!Notices_organization_level_7_key_fkey(*)
        """)

        if active:
            print("Applying active filter")
            query = query.eq("latest", True)
            current_time = datetime.now(timezone.utc).isoformat()
            print("Current time:", current_time)
            # Only include notices where the solicitation_response_deadline is in the future.
            query = query.gt("solicitation_response_deadline", current_time)

        # Apply include filters for NAICS codes.
        if include_naics:
            print("Including NAICS codes:", include_naics)
            query = query.in_("naics", include_naics)
        # Apply exclude filters for NAICS codes.
        if exclude_naics:
            for code in exclude_naics:
                query = query.neq("naics", code)

        # Apply filters for solicitation types.
        if include_solicitation_types:
            query = query.in_("type", include_solicitation_types)
        if exclude_solicitation_types:
            print("Excluding solicitation types: ", exclude_solicitation_types)
            for s_type in exclude_solicitation_types:
                query = query.neq("type", s_type)

        # Apply filters for PSC codes.
        if include_psc:
            query = query.in_("psc", include_psc)
        if exclude_psc:
            for psc in exclude_psc:
                query = query.neq("psc", psc)

         # Apply filters for set aside IDs from the joined setasides table.
        if include_set_aside_ids:
            # Convert provided set_aside_ids to integer type to match int8 column in database.
            include_set_aside_ids = [int(x) for x in include_set_aside_ids]
            print("Including set_aside_ids:", include_set_aside_ids)
            query = query.in_("setasides_details.set_aside_id", include_set_aside_ids)
        if exclude_set_aside_ids:
            # Convert provided set_aside_ids to integer type to match int8 column in database.
            exclude_set_aside_ids = [int(x) for x in exclude_set_aside_ids]
            print("Excluding set_aside_ids:", exclude_set_aside_ids)
            for set_aside_id in exclude_set_aside_ids:
                query = query.neq("setasides_details.set_aside_id", set_aside_id)
           

        # Apply an advanced keyword search on text fields.
        if keyword_query:
            or_filter = f"title.fts.{keyword_query},description_body.fts.{keyword_query}"
            query = query.or_(or_filter)

        # Apply pagination for the current batch.
        query = query.range(offset, offset + limit - 1)
        result = query.execute()

        # If no more data is returned, exit the loop.
        if not result.data:
            break

        all_notices.extend(result.data)
        offset += limit

    return all_notices

def main():
    load_dotenv()
    SUPABASE_URL = os.getenv("SUPABASE_URL")
    SUPABASE_KEY = os.getenv("SUPABASE_KEY")

    if not SUPABASE_URL or not SUPABASE_KEY:
        raise ValueError("SUPABASE_URL and SUPABASE_KEY environment variables must be set")

    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

    # Define filter parameters.
    active = True
    #active = False
    include_naics = [
        "237310", "488111", "493110", "518210", "541330", "541511", "541512", "541519",
        "541611", "541614", "562910", "513210", "516210", "561210", "221330", "236210",
        "236220", "237110", "237990", "238110", "238350", "238910", "238990", "333922",
        "541219", "541350", "541360", "541370", "541380", "541430", "541513", "541612",
        "541618", "541620", "541690", "541713", "541714", "541715", "541810", "541820",
        "541830", "541840", "541850", "541860", "541870", "541890", "541990", "561410",
        "561421", "561499", "561730", "561790", "611430"
    ]

    exclude_naics = []

    include_solicitation_types = []      # types to include
    #exclude_solicitation_types = []
    exclude_solicitation_types = ["s", "a", "u", "j", "l", "m", "g", "f"]  # types to exclude
    
    exclude_psc = []
    include_psc = []
    
    # Filters for set aside codes and IDs (from the joined setasides table).
    
    include_set_aside_ids = []  
    #exclude_set_aside_ids = []
    exclude_set_aside_ids = [16, 24, 17, 18, 19, 20, 22, 21, 23 ]  # example set_aside_id values to exclude

    keyword_query = None

    # Retrieve notices with the new filtering options.
    notices = get_filtered_notices(
        supabase=supabase,
        active=active,
        include_naics=include_naics,
        exclude_naics=exclude_naics,
        include_solicitation_types=include_solicitation_types,
        exclude_solicitation_types=exclude_solicitation_types,
        include_psc=include_psc,
        exclude_psc=exclude_psc,
        include_set_aside_ids=include_set_aside_ids,
        exclude_set_aside_ids=exclude_set_aside_ids,
        keyword_query=keyword_query
    )

    #print(json.dumps(notices, indent=2))
    print("Number of results: ", len(notices))
    with open("wnzTPS5NfNK5vVqRhbQ9i_results.json", "w") as f:
        json.dump(notices, f, indent=2, default=str)
    print("Saved results to wnzTPS5NfNK5vVqRhbQ9i_result.json file")

if __name__ == "__main__":
    main()

Applying active filter
Current time: 2025-03-17T20:19:25.867510+00:00
Including NAICS codes: ['237310', '488111', '493110', '518210', '541330', '541511', '541512', '541519', '541611', '541614', '562910', '513210', '516210', '561210', '221330', '236210', '236220', '237110', '237990', '238110', '238350', '238910', '238990', '333922', '541219', '541350', '541360', '541370', '541380', '541430', '541513', '541612', '541618', '541620', '541690', '541713', '541714', '541715', '541810', '541820', '541830', '541840', '541850', '541860', '541870', '541890', '541990', '561410', '561421', '561499', '561730', '561790', '611430']
Excluding solicitation types:  ['s', 'a', 'u', 'j', 'l', 'm', 'g', 'f']
Excluding set_aside_ids: [16, 24, 17, 18, 19, 20, 22, 21, 23]
Applying active filter
Current time: 2025-03-17T20:19:27.009057+00:00
Including NAICS codes: ['237310', '488111', '493110', '518210', '541330', '541511', '541512', '541519', '541611', '541614', '562910', '513210', '516210', '561210', '221330'

In [ ]:
import os
import re
import json
from typing import List, Dict, Optional
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime, timezone

def get_filtered_notices(
    supabase: Client,
    active: bool = True,
    include_naics: Optional[List[str]] = None,
    exclude_naics: Optional[List[str]] = None,
    include_solicitation_types: Optional[List[str]] = None,
    exclude_solicitation_types: Optional[List[str]] = None,
    include_psc: Optional[List[str]] = None,
    exclude_psc: Optional[List[str]] = None,
    include_set_aside_ids: Optional[List[str]] = None,  # new filter: include by set_aside_id
    exclude_set_aside_ids: Optional[List[str]] = None,  # new filter: exclude by set_aside_id
    include_organization_keys: Optional[List[str]] = None,  # new filter: include by organization_key (int8)
    exclude_organization_keys: Optional[List[str]] = None,  # new filter: exclude by organization_key (int8)
    keyword_query: Optional[str] = None
) -> List[Dict[str, any]]:
    """
    Retrieve rows from the 'notices' table applying the provided filters and paginating through all the available data.
    """
    all_notices = []
    limit = 1000  # Batch size for pagination.
    offset = 0

    while True:
        # Build the base query with embedded joins.
        query = supabase.from_("notices").select("""
            *,
            naics_details:naics!naics_id(*),
            psc_details:psc!psc_id(*),
            setasides_details:setasides!set_aside_id(*),
            solicitations_details:solicitations!solicitation_id(*),
            addresses_details:addresses!organization_address_key(*),
            organization_details:organizations!Notices_organization_key_fkey(*),
            organization_level_1_details:organizations!Notices_organization_level_1_key_fkey(*),
            organization_level_2_details:organizations!Notices_organization_level_2_key_fkey(*),
            organization_level_3_details:organizations!Notices_organization_level_3_key_fkey(*),
            organization_level_4_details:organizations!Notices_organization_level_4_key_fkey(*),
            organization_level_5_details:organizations!Notices_organization_level_5_key_fkey(*),
            organization_level_6_details:organizations!Notices_organization_level_6_key_fkey(*),
            organization_level_7_details:organizations!Notices_organization_level_7_key_fkey(*),
            solicitation_type_details:solicitation_types!type(*)
        """)

        if active:
            print("Applying active filter")
            query = query.eq("latest", True)
            current_time = datetime.now(timezone.utc).isoformat()
            print("Current time:", current_time)
            # Only include notices with a future solicitation_response_deadline.
            query = query.gt("solicitation_response_deadline", current_time)

        if include_naics:
            print("Including NAICS codes:", include_naics)
            query = query.in_("naics", include_naics)
        if exclude_naics:
            for code in exclude_naics:
                query = query.neq("naics", code)

        if include_solicitation_types:
            query = query.in_("type", include_solicitation_types)
        if exclude_solicitation_types:
            print("Excluding solicitation types:", exclude_solicitation_types)
            for s_type in exclude_solicitation_types:
                query = query.neq("type", s_type)

        if include_psc:
            query = query.in_("psc", include_psc)
        if exclude_psc:
            for psc in exclude_psc:
                query = query.neq("psc", psc)

        if include_set_aside_ids:
            # Convert set_aside_ids to integers for matching database type.
            include_set_aside_ids = [int(x) for x in include_set_aside_ids]
            print("Including set_aside_ids:", include_set_aside_ids)
            query = query.in_("setasides_details.set_aside_id", include_set_aside_ids)
        if exclude_set_aside_ids:
            # Convert set_aside_ids to integers for matching database type.
            exclude_set_aside_ids = [int(x) for x in exclude_set_aside_ids]
            print("Excluding set_aside_ids:", exclude_set_aside_ids)
            for set_aside_id in exclude_set_aside_ids:
                query = query.neq("setasides_details.set_aside_id", set_aside_id)
                
        if include_organization_keys:
            # Convert organization_keys to integers for matching database type.
            include_organization_keys = [int(x) for x in include_organization_keys]
            print("Including organization_keys:", include_organization_keys)
            query = query.in_("organization_details.organization_key", include_organization_keys)
        if exclude_organization_keys:
            # Convert organization_keys to integers for matching database type.
            exclude_organization_keys = [int(x) for x in exclude_organization_keys]
            print("Excluding organization_keys:", exclude_organization_keys)
            for org_key in exclude_organization_keys:
                query = query.neq("organization_details.organization_key", org_key)

        query = query.range(offset, offset + limit - 1)
        
        if keyword_query:
            print("Applying keyword search filter:", keyword_query)
            # Use full-text search on the "opportunity_text" field.
            query = query.text_search("opportunity_text", keyword_query, {'config': 'english', 'type': 'websearch'})
            
        result = query.execute()
        
        # Exit loop if no more data is returned.
        if not result.data:
            break

        all_notices.extend(result.data)
        offset += limit

    return all_notices

In [ ]:
def main():
    load_dotenv()
    SUPABASE_URL = os.getenv("SUPABASE_URL")
    SUPABASE_KEY = os.getenv("SUPABASE_KEY")

    if not SUPABASE_URL or not SUPABASE_KEY:
        raise ValueError("SUPABASE_URL and SUPABASE_KEY environment variables must be set")

    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

    # Define filter parameters.
    active = True
    include_naics = [
        "237310", "488111", "493110", "518210", "541330", "541511", "541512", "541519",
        "541611", "541614", "562910", "513210", "516210", "561210", "221330", "236210",
        "236220", "237110", "237990", "238110", "238350", "238910", "238990", "333922",
        "541219", "541350", "541360", "541370", "541380", "541430", "541513", "541612",
        "541618", "541620", "541690", "541713", "541714", "541715", "541810", "541820",
        "541830", "541840", "541850", "541860", "541870", "541890", "541990", "561410",
        "561421", "561499", "561730", "561790", "611430"
    ]
    exclude_naics = []

    include_solicitation_types = []  # types to include
    exclude_solicitation_types = ["s", "a", "u", "j", "l", "m", "g", "f"]  # types to exclude
    
    include_psc = []
    exclude_psc = []
    
    # Filters for set aside codes and IDs.
    include_set_aside_ids = []  
    exclude_set_aside_ids = [16, 24, 17, 18, 19, 20, 22, 21, 23]  # example set_aside_id values to exclude

    # New filters for organization keys (int8).
    include_organization_keys = []  # example organization keys to include (int8)
    exclude_organization_keys = []  # example organization keys to exclude (int8)

    keyword_query = "ITAD | media | destruction | digital | media | destruction"

    # Retrieve notices using the specified filters.
    notices = get_filtered_notices(
        supabase=supabase,
        active=active,
        include_naics=include_naics,
        exclude_naics=exclude_naics,
        include_solicitation_types=include_solicitation_types,
        exclude_solicitation_types=exclude_solicitation_types,
        include_psc=include_psc,
        exclude_psc=exclude_psc,
        include_set_aside_ids=include_set_aside_ids,
        exclude_set_aside_ids=exclude_set_aside_ids,
        include_organization_keys=include_organization_keys,
        exclude_organization_keys=exclude_organization_keys,
        keyword_query=keyword_query
    )

    print("Number of results:", len(notices))
    with open("wnzTPS5NfNK5vVqRhbQ9i_results.json", "w") as f:
        json.dump(notices, f, indent=2, default=str)
    print("Saved results to wnzTPS5NfNK5vVqRhbQ9i_result.json file")

if __name__ == "__main__":
    main()